In [35]:
!pip install --quiet transformers

In [36]:
import numpy as np
import torch
from typing import List
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSeq2SeqLM, PreTrainedTokenizer, pipeline
from transformers.modeling_outputs import Seq2SeqModelOutput

In [37]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [38]:
model_name = "yjernite/bart_eli5"

In [39]:
pos_tokenizer = AutoTokenizer.from_pretrained("vblagoje/roberta-base-english-upos")
pos_model = AutoModelForTokenClassification.from_pretrained("vblagoje/roberta-base-english-upos")

In [40]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
_ = model.to(device)

In [41]:
best_supporting_docs = ["when the skin is completely wet. The body continuously loses water by evaporation but the most significant amount of heat loss occurs during periods of increased physical activity. Evaporative cooling Evaporative cooling happens when water vapor is added to the surrounding air. The energy needed to evaporate the water is taken from the air in the form of sensible heat and converted into latent heat, while the air remains at a constant enthalpy. Latent heat describes the amount of heat that is needed to evaporate the liquid; this heat comes from the liquid itself and the surrounding gas and surfaces.",
                "at greater pressures. There is an ambiguity, however, as to the meaning of the terms 'heating' and 'cooling' in Sandstrom's theorem. So far, heating and cooling has always been interpreted in the literature as being associated with 'surface heating' and 'surface cooling' respectively. In real fluids, however, molecular and turbulent diffusion always cause internal heating/cooling even in absence of external heating/cooling, as long as the temperature of the fluid considered is non-uniform. As is well-known, molecular and turbulent diffusion tends to relax the system toward thermodynamic equilibrium, i.e., toward an isothermal state, which for a statically stable fluid, will warm up",
                "are not in a relation of thermal equilibrium, heat will flow from the hotter to the colder, by whatever pathway, conductive or radiative, is available, and this flow will continue until thermal equilibrium is reached and then they will have the same temperature. One form of thermal equilibrium is radiative exchange equilibrium. Two bodies, each with its own uniform temperature, in solely radiative connection, no matter how far apart, or what partially obstructive, reflective, or refractive, obstacles lie in their path of radiative exchange, not moving relative to one another, will exchange thermal radiation, in net the hotter transferring energy to",
                "air condition and moving along a line of constant enthalpy toward a state of higher humidity. A simple example of natural evaporative cooling is perspiration, or sweat, secreted by the body, evaporation of which cools the body. The amount of heat transfer depends on the evaporation rate, however for each kilogram of water vaporized 2,257 kJ of energy (about 890 BTU per pound of pure water, at 95 °F (35 °C)) are transferred. The evaporation rate depends on the temperature and humidity of the air, which is why sweat accumulates more on humid days, as it does not evaporate fast enough. Vapor-compression refrigeration uses evaporative cooling,",            
                "Thermal contact conductance In physics, thermal contact conductance is the study of heat conduction between solid bodies in thermal contact. The thermal contact conductance coefficient, , is a property indicating the thermal conductivity, or ability to conduct heat, between two bodies in contact. The inverse of this property is termed thermal contact resistance. Factors influencing contact conductance Thermal contact conductance is a complicated phenomenon, influenced by many factors. Experience shows that the most important ones are as follows: Contact pressure For thermal transport between two contacting bodies, such as particles in a granular medium, the contact pressure is the factor"]

worst_supporting_docs = ["to completely boil or vaporize water (what is known as enthalpy of vaporization) is roughly 540 times that required for a one-degree increase. Water's sizable enthalpy of vaporization is why one's skin can be burned so quickly as steam condenses on it (heading from red to green in Fig. 7 above). In the opposite direction, this is why one's skin feels cool as liquid water on it evaporates (a process that occurs at a sub-ambient wet-bulb temperature that is dependent on relative humidity). Water's highly energetic enthalpy of vaporization is also an important factor underlying why solar pool covers (floating, insulated blankets that",
                         "and this is because temperature is an intensive variable. Bodies in thermodynamic equilibrium For experimental physics, hotness means that, when comparing any two given bodies in their respective separate thermodynamic equilibria, any two suitably given empirical thermometers with numerical scale readings will agree as to which is the hotter of the two given bodies, or that they have the same temperature. This does not require the two thermometers to have a linear relation between their numerical scale readings, but it does require that the relation between their numerical readings shall be strictly monotonic. A definite sense of greater hotness can",
                         "phase of atmospheric or ocean water, vaporization, condensation, freezing or melting, whereas sensible heat is energy transferred that is evident in change of the temperature of the atmosphere or ocean, or ice, without those phase changes, though it is associated with changes of pressure and volume. The original usage of the term, as introduced by Black, was applied to systems that were intentionally held at constant temperature. Such usage referred to latent heat of expansion and several other related latent heats. These latent heats are defined independently of the conceptual framework of thermodynamics. When a body is heated at constant temperature",
                         "Sensible heat Sensible heat is heat exchanged by a body or thermodynamic system in which the exchange of heat changes the temperature of the body or system, and some macroscopic variables of the body or system, but leaves unchanged certain other macroscopic variables of the body or system, such as volume or pressure.",
                         "changes. Conduction On a microscopic scale, heat conduction occurs as hot, rapidly moving or vibrating atoms and molecules interact with neighboring atoms and molecules, transferring some of their energy (heat) to these neighboring particles. In other words, heat is transferred by conduction when adjacent atoms vibrate against one another, or as electrons move from one atom to another. Conduction is the most significant means of heat transfer within a solid or between solid objects in thermal contact. Fluids—especially gases—are less conductive. Thermal contact conductance is the study of heat conduction between solid bodies in contact. The process of heat transfer"]


unrelated_supporting_docs = ["from competition, lose a maximum of body fat (referred to as \"cutting\") while preserving as much muscular mass as possible The bulking phase entails remaining in a net positive energy balance (calorie surplus) The amount of a surplus in which a person remains is based on the person's goals, as a bigger surplus and longer bulking phase will create more fat tissue The surplus of calories relative to one's energy balance will ensure that muscles remain in a state of anabolism The cutting phase entails remaining in a net negative energy balance (calorie deficit). The main goal of cutting is to",
                             "and decreased body fat, blood pressure, LDL cholesterol and triglycerides The body's basal metabolic rate increases with increases in muscle mass, which promotes long-term fat loss and helps dieters avoid yo-yo dietin Moreover, intense workouts elevate metabolism for several hours following the workout, which also promotes fat loss Weight training also provides functional benefits. Stronger muscles improve posture, provide better support for joints, and reduce the risk of injury from everyday activities. Older people who take up weight training can prevent some of the loss of muscle tissue that normally accompanies aging\u2014and even regain some functional strength\u2014and by doing so, become less",
                             "workouts the day after interval sessions. Finally the race phase of the periodization approach is where the duration of the workouts decreases but intense workouts remain so as to keep the high lactate threshold that was gained in the build phase In Ironman training, the race phase is where a long \"taper\" occurs of up to 4 weeks for highly trained Ironman racers  A final phase is designated transition and is a period of time, where the body is allowed to recover from the hard race effort and some maintenance endurance training is performed so the high fitness level",
                             "Spot reduction Spot reduction refers to the claim that fat in a certain area of the body can be targeted for reduction through exercise of specific muscles in that desired area For example, exercising the abdominal muscles in an effort to lose weight in or around one's midsection History Over time it has become clear to trainers and health professionals that the idea of working out certain muscle groups, in order to burn fat in that specific area, is not possible. Advertisements, magazines, internet trainers, and social media continue to push the idea that through exercising small isolated muscles, such",
                             "can be shown more easily in lean people Those who are overweight have a better chance of reducing fat if they increase their overall fitness levels This can be achieved by doing both High-intensity interval training cardio and, resistance training that focuses on gaining muscle mass in both large and small muscle groups This increase in muscle mass will create a larger need for energy that comes from fatty acids in stored fat cells Myth Although professionals know spot reduction is a myth, many people still believe that it is possible to choose where fat can be lost on one's"]

# !!!IMPORTANT!!!
# choose either best_supporting_docs or worst_supporting_docs returned by the retriever
supporting_docs = best_supporting_docs

# this is the golden answer to our question
answer = "Water transfers heat more efficiently than air. When something feels cold it's because heat is being transferred from your skin to whatever you're touching. Since water absorbs the heat more readily than air, it feels colder."

In [42]:
def prepare_model_input(tokenizer, question, supporting_docs):
  """
  Concatenate question and support document into BART ELI5 model input format
  and return encoded model input
  """
  conditioned_doc = "<P> " + " <P> ".join([doc for doc in supporting_docs])  
  query_and_docs = "question: {} context: {}".format(question, conditioned_doc)
  query_and_docs_encoded = tokenizer([(query_and_docs, "A")], truncation=True, padding=True, return_tensors="pt")
  return query_and_docs_encoded

In [43]:
def get_token_probs(tokenizer: PreTrainedTokenizer, output: Seq2SeqModelOutput, answer_tokens: List[int]):
  """
  Returns dictionary with three lists: tokens, token_probs, and topk_preds 

  Tokens represent tokens in the LFQA answer, token_probs are probabilities token t is generated at the particular 
  token position in the answer, while topk_preds are LM topk predictions for each token t position in the answer
  """
  token_probs = {"tokens": [], "token_probs":[], "topk_preds": []}
  for i, token_logits in enumerate(output.logits[0]):
    probs = torch.nn.functional.softmax(token_logits, dim=0)  
    token_prob = probs[answer_tokens[i]].item()
    _, preds = torch.topk(probs, k=5)
    topk_preds = tokenizer.decode(preds).split()
    token = tokenizer.decode(answer_tokens[i])

    token_probs["tokens"].append(token)
    token_probs["token_probs"].append(token_prob)
    token_probs["topk_preds"].append(topk_preds)    
  return token_probs  

In [44]:
def fill_mask_fast(model, tokenizer, masked_doc: str):
  """
  Fill mask in masked_doc with top model predictions
  """
  input = tokenizer(masked_doc, return_tensors="pt")
  mask_token_index = torch.where(input["input_ids"] == tokenizer.mask_token_id)[1]
  output = model(**input)
  mask_token_logits = output.logits[0, mask_token_index, :]
  
  top_5_tokens = torch.topk(mask_token_logits, 5, dim=1)
  # at indices where the mask tokens are replace current values with model output top topk=1
  return torch.scatter(input["input_ids"][0], 0, mask_token_index, top_5_tokens.indices[:,0]).unsqueeze(dim=0)
  

In [45]:
def print_token_probs(output_probs):
  print ("{:<8} {:<15} {:<30}".format('prob','token','top_5'))
  for k,v,q in zip(output_probs["token_probs"], output_probs["tokens"], output_probs["topk_preds"]):   
    print ("{:<8} {:<15} {:<30}".format( round(k,2), v.strip(), str(q)))

In [46]:
q = "Why does water heated to room temperature feel colder than the air around it?"
query_and_docs_encoded = prepare_model_input(tokenizer, q, supporting_docs)

In [47]:
decoder_input = tokenizer(answer, return_tensors='pt')
output = model(input_ids=query_and_docs_encoded["input_ids"].to(device), 
               attention_mask=query_and_docs_encoded["attention_mask"].to(device), 
               labels=decoder_input['input_ids'].to(device))

In [48]:
doc_probs = get_token_probs(tokenizer, output, decoder_input["input_ids"].squeeze().tolist())

In [49]:
print_token_probs(doc_probs)

prob     token           top_5                         
0.0      <s>             ['Water', 'It', 'The', 'Because', 'I']
0.0      Water           ['Water', 'It', 'The', 'Because', 'I']
0.02     transfers       ['is', 'has', 'doesn', 'takes', 'evapor']
0.84     heat            ['heat', 'its', 'the', 'energy', 'body']
0.02     more            ['away', 'from', 'faster', 'very', 'to']
0.5      efficiently     ['efficiently', 'quickly', 'effectively', 'slowly', 'easily']
0.75     than            ['than', 'at.,', 'to']        
0.91     air             ['air', 'the', 'your', 'any', 'a']
0.43     .               ['.,', 'does', 'so', 'at']    
0.07     When            ['It', 'The', 'Water', 'Your', 'When']
0.01     something       ['you', 'water', 'it', 'the', 'air']
0.03     feels           ['is', 'transfers', 'absorbs', 'cool', 'feels']
0.33     cold            ['colder', 'cold', 'cooler', '"', 'hot']
0.18     it              [',', 'it', 'to', 'you', 'your']
0.38     's              ["is's", '

In [50]:
def create_token_mask_list(model, tokenizer, target_text: str, pos_tag_filter:List[str]):
  """
  From target_text creates a list of tokens to mask given list of POS tags to include
  """
  tokens_to_mask = []
  pos_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer)
  target_text_pos = pos_pipe(target_text)
  for a_pos in target_text_pos:
    if a_pos["entity"] in pos_tag_filter:
      tokens_to_mask.append(a_pos["word"]) 
  return tokens_to_mask        

In [51]:
def mask_document(model, tokenizer, document: str, tokens_to_mask: List[str]):
  """
  Masks tokens_to_mask in the given document
  Returns document with <mask> tokens replacing any of the tokens_to_mask that were found
  """
  document_indices_to_mask = []
  pos_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer)
  document_pos = pos_pipe(document)
  for d_pos in document_pos:
    if d_pos["word"] in tokens_to_mask:
      document_indices_to_mask.append(d_pos["index"])      
  

  encoded_doc = tokenizer(document, return_tensors="pt")
  encoded_doc = encoded_doc["input_ids"].squeeze()  
  mask_tokens = torch.ones(len(document_indices_to_mask), dtype=encoded_doc.dtype) * tokenizer.mask_token_id
  doc_masked_encoded = torch.scatter(encoded_doc, 0, torch.tensor(document_indices_to_mask, dtype=encoded_doc.dtype), mask_tokens)
  doc_with_mask = tokenizer.decode(doc_masked_encoded)
  doc_with_mask = doc_with_mask.replace("<s>", "")
  doc_with_mask = doc_with_mask.replace("</s>", "")
  return doc_with_mask.strip()


In [52]:
universal_pos_tags = ['ADJ','ADP','ADV', 'AUX', 'CCONJ','DET','INTJ','NOUN','NUM','PART','PRON','PROPN','PUNCT','SCONJ','SYM','VERB','X']
### The universal part-of-speech (pos) categories can be refered to https://universaldependencies.org/u/pos/, which include:
# ADJ: adjective
# ADP: adposition
# ADV: adverb
# AUX: auxiliary
# CCONJ: coordinating conjunction
# DET: determiner
# INTJ: interjection
# NOUN: noun
# NUM: numeral
# PART: particle
# PRON: pronoun
# PROPN: proper noun
# PUNCT: punctuation
# SCONJ: subordinating conjunction
# SYM: symbol
# VERB: verb
# X: other
unimportant_pos_tags = ['PUNCT', 'SYM', 'DET', 'PART', 'CCONJ', 'SCONJ', "ADP", "ADJ", "X", "AUX"]
important_pos_tags = [tag for tag in universal_pos_tags if tag not in unimportant_pos_tags]


In [53]:
important_pos_tags

['ADV', 'INTJ', 'NOUN', 'NUM', 'PRON', 'PROPN', 'VERB']

In [54]:
masked_token_list = create_token_mask_list(pos_model, pos_tokenizer, answer, important_pos_tags)
masked_supporting_docs = [mask_document(pos_model, pos_tokenizer, doc, masked_token_list) for doc in supporting_docs]

In [55]:
# fill-the-mask for the mask tokens 
unmasked_supporting_docs = [fill_mask_fast(model, tokenizer, doc) for doc in masked_supporting_docs]
masked_supporting_docs_solved = [tokenizer.batch_decode(d, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0].strip() for d in unmasked_supporting_docs]

In [56]:
# compare masked source doc, mask-fill source doc, and unmasked doc
for k,v,q in zip(supporting_docs, masked_supporting_docs, masked_supporting_docs_solved):
  print(k)
  print(v)
  print(q)
  print("------------------------------\n")

when the skin is completely wet. The body continuously loses water by evaporation but the most significant amount of heat loss occurs during periods of increased physical activity. Evaporative cooling Evaporative cooling happens when water vapor is added to the surrounding air. The energy needed to evaporate the water is taken from the air in the form of sensible heat and converted into latent heat, while the air remains at a constant enthalpy. Latent heat describes the amount of heat that is needed to evaporate the liquid; this heat comes from the liquid itself and the surrounding gas and surfaces.
when the<mask> is completely wet<mask> The body continuously loses<mask> by evaporation but the most significant amount of<mask> loss occurs during periods of increased physical activity<mask> Evaporative cooling Evaporative cooling happens when<mask> vapor is added to the surrounding<mask><mask> The energy needed to evaporate the<mask> is taken from the<mask> in the form of sensible<mask> 

In [57]:
masked_supporting_docs_solved

['when the body is completely wet, The body continuously loses heat by evaporation but the most significant amount of loss loss occurs during periods of increased physical activity. Evaporative cooling Evaporative cooling happens when the vapor is added to the surrounding air. The energy needed to evaporate the vapor is taken from the surrounding in the form of sensible heat and converted into latent latent, while the vapor remains at a constant enthalpy. Latent cooling describes the amount of energy that is needed to evaporate the liquid; this is comes from the liquid itself and the surrounding gas and surfaces.',
 "at greater pressures the There is an ambiguity, however, as to the meaning of the terms 'heating' and 'cooling' in Sandstrom's theorem theorem So far, heating and cooling has always been interpreted in the literature as being associated with'surface heating' and'surface cooling' respectively. In real fluids, however, molecular and turbulent diffusion always cause internal 

In [58]:
query_and_docs_encoded = prepare_model_input(tokenizer, q, masked_supporting_docs_solved)

In [59]:
# feed mask-fill source doc along with unchanged answer
decoder_input = tokenizer(answer, return_tensors='pt')
output_masked_doc = model(input_ids=query_and_docs_encoded["input_ids"], 
                          attention_mask=query_and_docs_encoded["attention_mask"], 
                          labels=decoder_input['input_ids'])

In [60]:
# get the probs for the masked source answer pair
masked_doc_probs = get_token_probs(tokenizer, output_masked_doc, decoder_input["input_ids"].squeeze().tolist())

In [61]:
print_token_probs(masked_doc_probs)

prob     token           top_5                         
0.0      <s>             ['The', 'It', 'When', 'Thermal', 'If']
0.0      Water           ['The', 'It', 'When', 'Thermal', 'If']
0.01     transfers       ['conducts', 'is', 'and', 'has', 'absorbs']
0.9      heat            ['heat', 'energy', 'its', 'a', 'the']
0.03     more            ['very', 'better.', 'faster', 'through']
0.38     efficiently     ['efficiently', 'quickly', 'easily', 'effectively', 'slowly']
0.53     than            ['than', 'when', 'through', 'at.']
0.31     air             ['air', 'solid', 'a', 'the', 'other']
0.43     .               ['.,', 'does', 'or', 'at']    
0.06     When            ['The', 'So', 'If', 'When', 'It']
0.02     something       ['you', 'two', 'water', 'a', 'the']
0.0      feels           ['is', 'like', 'cool', 'touches', 'has']
0.16     cold            ['cold', 'hot', 'like', 'warm', 'colder']
0.14     it              [',', 'it', 'or', 'and', 'to']
0.14     's              ["is's", 'absorbs'

In [62]:
def merge_subwords(subwords, summ_scores, mask_scores):
    
    assert len(subwords) == len(summ_scores)
    assert len(mask_scores) == len(summ_scores)

    merge_word = list()
    merge_summ_score = list()
    merge_mask_score = list()
    cur_word = list()
    cur_summ_score = list()
    cur_mask_score = list()
    for subword, summ_score, mask_score in zip(subwords, summ_scores, mask_scores):
        if subword in ['', ' ', '<s>', '</s>']:
            continue

        if subword.startswith(' ') and len(cur_word) > 0:
            merge_word.append(''.join(cur_word).lstrip())
            merge_summ_score.append(cur_summ_score[0])
            merge_mask_score.append(cur_mask_score[0])
            cur_word.clear()
            cur_summ_score.clear()
            cur_mask_score.clear()

        cur_word.append(subword)
        cur_summ_score.append(summ_score)
        cur_mask_score.append(mask_score)

    if len(cur_word) != 0:
        merge_word.append(''.join(cur_word).lstrip())
        merge_summ_score.append(cur_summ_score[0])
        merge_mask_score.append(cur_mask_score[0])

    return merge_word, merge_summ_score, merge_mask_score   

In [63]:
masked_token_list = [word.replace("Ġ", "") for word in masked_token_list if len(word)>2]

In [64]:
masked_token_list = set(masked_token_list)

In [65]:
tokenized_answer = doc_probs["tokens"]
merge_tokens, unmasked_score, masked_score = merge_subwords(tokenized_answer, doc_probs["token_probs"], masked_doc_probs["token_probs"])

# calculate coco scores by subtracting summ_core from mask_score
scores = [unmasked_score[idx]-masked_score[idx] for idx in range(len(merge_tokens)) if merge_tokens[idx] in masked_token_list]

In [66]:
#
#masked_token_list, merge_tokens

In [67]:
# and finally coco score itself
coco_score = np.mean(scores)

In [68]:
coco_score

0.09018472499413197